# Generation du corpus en texte brut

## Constantes

In [1]:
CHECKSUM_PATH = "./checksum-xmls.csv"
CORPORA_PATH = "../../data/raw/corpora/**/*"
WRITE_TEXT = True # Set to True for really applying XSL
FORCE_REGENERATE = False  # Mostly useful if the XSL changes
FORCE_GENERATE_CSV_FROM_SCRATCH = False # Force writing from scratch the CHECKSUM

import tqdm

## Generateur de XSLT à la volée

In [2]:
import lxml.etree as ET

def get_xslt(citation_scheme):
    additional_xsl = ""
    for level in range(len(citation_scheme)):
        xpath = f"{citation_scheme[level].scope}{citation_scheme[level].xpath}".replace("='?'", "").replace("=\"?\"", "")
        name = citation_scheme[level].name or "unknown"
        if level == 0:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            [REF:{name}.<xsl:value-of select="@n"/>]
            <xsl:apply-templates>
                <xsl:with-param name="previous"><xsl:value-of select="@n"/></xsl:with-param>
            </xsl:apply-templates>
        </xsl:template>"""
        elif level == len(citation_scheme) - 1:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            <xsl:param name="previous" />
            [REF:{name}.<xsl:value-of select="$previous" />.<xsl:value-of select="@n"/>]
            <xsl:apply-templates />
        </xsl:template>"""
            # LAST LEVEL
        else:
            additional_xsl += "\n" + f"""
        <xsl:template match="{xpath}">
            <xsl:param name="previous" />
            [REF:{name}.<xsl:value-of select="$previous" />.<xsl:value-of select="@n"/>]
            <xsl:apply-templates>
                <xsl:with-param name="previous"><xsl:value-of select="$previous" />.<xsl:value-of select="@n"/></xsl:with-param>
            </xsl:apply-templates>
        </xsl:template>"""
    return ET.XSLT(ET.fromstring("""<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
    xmlns:xs="http://www.w3.org/2001/XMLSchema"
    exclude-result-prefixes="xs" xmlns:tei="http://www.tei-c.org/ns/1.0"
    version="1.0">
    <xsl:output method="text" encoding="UTF-8"/>
    <xsl:template match="tei:note">
        <xsl:text> </xsl:text>
    </xsl:template>
    <xsl:template match="tei:p|tei:div|tei:seg|tei:l|tei:w|tei:ab">
        <xsl:param name="previous" />
        <xsl:text> </xsl:text><xsl:apply-templates ><xsl:with-param name="previous" select="$previous" /></xsl:apply-templates>
    </xsl:template>
    <xsl:template match="tei:listPerson|tei:teiHeader|tei:label|tei:ref|tei:milestone|tei:orig|tei:abbr|tei:head|tei:title|tei:teiHeader|tei:del|tei:g|tei:bibl|tei:front|tei:back|tei:foreign[@xml:lang!='grc']" />
    {previous}
    <xsl:template match="*[@xml:lang='eng']"/>
    <xsl:template match="tei:TEI[@xml:lang='eng']"><xsl:apply-templates /></xsl:template>
    <xsl:template match="*[@type='edition' and @xml:lang='eng']"><xsl:apply-templates /></xsl:template>
</xsl:stylesheet>""".format(previous=additional_xsl)))

## Lecture des dépôts

In [3]:
import glob
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver


repositories = list(glob.glob(CORPORA_PATH, recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/lascivaroma_additional-texts/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is 

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0917/phi001/phi0917.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0119/phi004/phi0119.phi

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi004/phi1351.phi004.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


## Créations des dossiers

In [4]:
import os

os.makedirs("./lemmatized/plain-text/", exist_ok=True)

## Ouverture de l'XSL

In [5]:
import lxml.etree as ET

with open("../../helpers/reader/passage.transform.xsl") as tr:
    xml = ET.parse(tr)
    xsl = ET.XSLT(xml)

## Vérification des changements de sources

Currently needs to install following version: 
```shell
pip install --upgrade https://github.com/PonteIneptique/pie/archive/update-pie/1.6.0.zip#egg=nlp_pie 
pip install --upgrade https://github.com/hipster-philology/nlp-pie-taggers/archive/feature/hashlist.zip#egg=pie_extended
```

### Calcul des hashes

In [6]:
from hash_compute import md5sum

xml_current_hashes = {
    
}

for text in tqdm.tqdm(resolver.texts):
    if text.lang == "lat":
        try:
            depth = len(text.citation)
            xml_current_hashes[os.path.abspath(text.path)] = md5sum(text.path)
        except Exception as E:
            print(E, text.id)

100%|██████████| 969/969 [00:00<00:00, 2453.10it/s]


### Ouverture des checksums de source enregistrées

In [7]:
from hash_compute import check_checksum_from_file, read_checksum_csv

former_checksums = read_checksum_csv(CHECKSUM_PATH)
former_sources = [
    infos.source
    for infos in former_checksums.values()
    if infos.source and os.path.exists(infos.source)
]
deleted_sources = {
    target: infos.source
    for target, infos in former_checksums.items()
    if infos.source and not os.path.exists(infos.source)
}

_, changed_sources = check_checksum_from_file(CHECKSUM_PATH)

new_sources = [
    source
    for source in xml_current_hashes
    if source not in former_sources
]

print(f"{len(former_sources)} former source file")
print(f"{len(new_sources)} new source file detected")
print(f"{len(changed_sources)} changed source files detected")
print(f"{len(deleted_sources)} deleted source files detected")

for source in deleted_sources:
    os.remove(source)
    print(f"Removed {source}")
    


841 former source file
12 new source file detected
5 changed source files detected
0 deleted source files detected


### Choix entre re-XSLisé tout ou seulement les éléments demandés

In [8]:
texts_to_generate = []

if FORCE_REGENERATE:
    texts_to_generate = list(xml_current_hashes.keys())
else:
    texts_to_generate = new_sources + changed_sources
texts_to_generate

['/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0039c/stoa001/stoa0039c.stoa001.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0030/stoa008/stoa0030.stoa008.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0012a/stoa002/stoa0012a.stoa002.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0020b/stoa002/stoa0020b.stoa002.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0030b/stoa001/stoa0030b.stoa001.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0045/stoa003a/stoa0045.stoa003a.digilibLT-lat1.xml',
 '/home/thibault/dev/these/data/raw/corpora/lascivaroma_digiliblt/lascivaroma_digiliblt/data/stoa0045/

## Conversion des XML en plein texte

In [9]:
import tqdm

sources_to_out = {
    
}
import regex

WRITE_TEXT = True
notWord = regex.compile("\W+")
space = regex.compile("\s+")
for text in tqdm.tqdm(resolver.texts):
    # If it's a Latin text and it's marked as a text to regenerate
    if text.lang == "lat" and os.path.abspath(text.path) in texts_to_generate:
        try:
            on_site_xsl = """"""
            main_text = resolver.getTextualNode(textId=text.id)
            out_path = "./lemmatized/plain-text/{}.txt".format(text.id)
            xslt = get_xslt(main_text.citation)
            sources_to_out[os.path.abspath(text.path)] = os.path.abspath(out_path)
            if WRITE_TEXT:
                with open(out_path, "w") as out:
                    out.write(space.sub(" ", str(xslt(main_text.xml))).replace("[REF", "\n[REF"))
        except Exception as E:
            print(text, E)
            raise E
            
print(f"{len(sources_to_out)} regenerated files")

100%|██████████| 969/969 [00:00<00:00, 3886.54it/s]

17 regenerated files


## Compte des mots

In [10]:
cnt = 0
for text in glob.glob("lemmatized/plain-text/*.txt"):
    if "-pie" not in text:
        with open(text) as f:
            cnt += len(f.read().split())
print(cnt)

18266713


## Génération ou mise à jour du fichier sources -> hash

In [11]:
import csv
if os.path.isfile(CHECKSUM_PATH) and FORCE_GENERATE_CSV_FROM_SCRATCH != True:
    rows = [
        ["input","checksum","source","source_checksum"]
    ]
    for text_file, checksuminfo in former_checksums.items():
        if checksuminfo.source in sources_to_out:
            rows.append([
                text_file,
                checksuminfo.checksum,
                checksuminfo.source,
                md5sum(checksuminfo.source)
            ])
            del sources_to_out[checksuminfo.source]
            # Update the checksum of the source BUT NOT THE TEXTFILE
        else:
            rows.append([
                text_file,
                checksuminfo.checksum,
                checksuminfo.source,
                checksuminfo.source_checksum
            ])# Write the current line as is.
    # And finally, we write new ones with new sources
    for source_file, output_file in sources_to_out.items():
        rows.append([
            output_file,
            "need_lemmatization",
            source_file,
            md5sum(source_file)
        ])
else:
    # Doing from scratch, we recompute everything:
    full = {
        source: glob.glob(
            "./lemmatized/plain-text/*{}".format(os.path.basename(source).replace(".xml", ".txt"))
        )[0]
        for source, md5 in xml_current_hashes.items()
    }
    
    def new_file(source, sources_to_out):
        """ Check if the file was newly lemmatized """
        if source in sources_to_out:
            return "need_lemmatization"
        else:
            return md5sum(source)
        
    rows = [
        ["input","checksum","source","source_checksum"]
    ] + [
        [
            plaintext,
            new_file(source, sources_to_out),
            source,
            md5sum(source)
        ]
        for source, plaintext in full.items()
    ]
 
with open(CHECKSUM_PATH, "w") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

In [12]:
!zip lemmatized/plain-text.zip lemmatized/plain-text -r

updating: lemmatized/plain-text/ (stored 0%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi038.perseus-lat1.txt (deflated 64%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi2331.phi023.perseus-lat2.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0022.stoa034.opp-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0119.phi006.perseus-lat2.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0959.phi003.perseus-lat2.txt (deflated 55%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa016.opp-lat1.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0167.stoa001.digilibLT-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0187b.stoa001.digilibLT-lat1.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi043.perseus-lat1.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0104p.stoa010.opp-lat1.txt (defla

updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi037.perseus-lat1.txt (deflated 63%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0255.stoa011.perseus-lat2.txt (deflated 57%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0233h.stoa001.opp-lat1.txt (deflated 57%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0959.phi010.perseus-lat2.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1351.phi001.perseus-lat1.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0208a.stoa001.digilibLT-lat1.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0255.stoa004.perseus-lat2.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1263.phi001.corpuscorporum-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa030.opp-lat2.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0195a.stoa002.opp-lat1.txt (deflated 56%)
updating: lemmatized/plain-t

updating: lemmatized/plain-text/urn:cts:latinLit:stoa0249a.stoa002.opp-lat1.txt (deflated 63%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0238.stoa001.perseus-lat2.txt (deflated 64%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi046.perseus-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa008.opp-lat2.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0040.stoa077d.opp-lat1.txt (deflated 55%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0121g.stoa005.opp-lat1.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0171.stoa009.opp-lat1.txt (deflated 63%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0255.stoa010.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0089.stoa013.perseus-lat2.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa017.opp-lat2.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:lat

updating: lemmatized/plain-text/urn:cts:latinLit:phi1351.phi004.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0690.phi003.perseus-lat2.txt (deflated 63%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0040.stoa021.opp-lat1.txt (deflated 66%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0959.phi009.perseus-lat2.txt (deflated 64%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0040.stoa056.opp-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0045.stoa028.perseus-lat2.txt (deflated 56%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi015.perseus-lat2.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0114a.stoa011.opp-lat1.txt (deflated 53%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0171.stoa007.opp-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1017.phi006.perseus-lat2.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latin

updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119.stoa001.opp-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0104p.stoa015.opp-lat1.txt (deflated 57%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0089.stoa009.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa024.opp-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0058.stoa023.perseus-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0245c.stoa001.opp-lat1.txt (deflated 48%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi006.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0215b.stoa003.opp-lat1.txt (deflated 64%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1014.phi003.perseus-lat1.txt (deflated 59%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0022.stoa035.opp-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latin

updating: lemmatized/plain-text/urn:cts:latinLit:stoa0058.stoa001.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi2331.phi029.perseus-lat2.txt (deflated 55%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0631.phi001.perseus-lat3.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0054.stoa001a.opp-lat1.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa021.opp-lat1.txt (deflated 59%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0117p.stoa001.opp-lat1.txt (deflated 56%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1351.phi002.perseus-lat1.txt (deflated 59%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0045.stoa004.perseus-lat2.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0134.phi006.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1017.phi002.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:

updating: lemmatized/plain-text/urn:cts:greekLit:tlg0527.tlg004.obi-lat1.txt (deflated 71%)
updating: lemmatized/plain-text/urn:cts:greekLit:tlg0527.tlg016.obi-lat1.txt (deflated 68%)
updating: lemmatized/plain-text/urn:cts:greekLit:tlg0527.tlg044.obi-lat1.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:greekLit:tlg0031.tlg011.obi-lat1.txt (deflated 62%)
updating: lemmatized/plain-text/urn:cts:greekLit:tlg0527.tlg045.obi-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0234b.stoa002.digilibLT-lat1.txt (deflated 53%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0234b.stoa001.digilibLT-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1348.phi002.thayer-lat1.txt (deflated 54%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi1348.phi005a.thayer-lat1.txt (deflated 55%)
updating: lemmatized/plain-text/urn:cts:latinLit:phi0474.phi031.perseus-lat2.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:ph

updating: lemmatized/plain-text/urn:cts:latinLit:stoa0022.stoa054.corpuscorporum-lat1.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0318.stoa002.digilibLT-lat1.txt (deflated 58%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0365.stoa002.digilibLT-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0162.stoa023.corpuscorporum-lat1.txt (deflated 65%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119a.stoa001b.digilibLT-lat1.txt (deflated 61%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0271.stoa001.digilibLT-lat1.txt (deflated 63%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119a.stoa001c.digilibLT-lat1.txt (deflated 60%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119a.stoa001f.digilibLT-lat1.txt (deflated 59%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119a.stoa001e.digilibLT-lat1.txt (deflated 59%)
updating: lemmatized/plain-text/urn:cts:latinLit:stoa0119a.stoa001d.digilibLT-lat1